In [7]:
import torch
import torchvision
import torch.nn
import torch.optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score


# Check for available device (GPU, MPS, or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:

from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class FruitDataset(Dataset):
    def __init__(self, root_dir, classes, transform=None):
        self.root_dir = root_dir
        self.classes = classes
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Load all images and labels
        for label, fruit_class in enumerate(classes):
            class_dir = os.path.join(root_dir, fruit_class)
            for img_file in os.listdir(class_dir):
                if img_file.endswith(('.png', '.jpg', '.jpeg')):
                    self.images.append(os.path.join(class_dir, img_file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [ ]:

# Define the classes (categories) in the dataset
classes = ['tomato', 'cherry', 'strawberry']


In [ ]:

from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [ ]:

from torch.utils.data import random_split

# Dynamically calculate train and validation sizes based on actual dataset length
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Preprocessed dataset split with dynamic length calculation
preprocessed_dataset = FruitDataset(root_dir='processed_data', classes=classes, transform=transform)
train_size_preprocessed = int(0.8 * len(preprocessed_dataset))
val_size_preprocessed = len(preprocessed_dataset) - train_size_preprocessed
train_dataset_preprocessed, val_dataset_preprocessed = random_split(preprocessed_dataset, [train_size_preprocessed, val_size_preprocessed])


In [ ]:

import os
import numpy as np
from PIL import Image, ImageOps
import cv2

# Define dataset paths
dataset_path = 'train_data'
output_path = 'processed_data'
classes = ['tomato', 'cherry', 'strawberry']
target_size = (256, 256)

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)
for cls in classes:
    os.makedirs(os.path.join(output_path, cls), exist_ok=True)

# Function to detect blur
def is_blurry(image):
    gray_image = np.array(image.convert('L'))
    laplacian_var = cv2.Laplacian(gray_image, cv2.CV_64F).var()
    return laplacian_var < 100  # Threshold for blurriness

# Preprocess images (without edge detection)
for fruit_class in classes:
    folder_path = os.path.join(dataset_path, fruit_class)
    output_folder_path = os.path.join(output_path, fruit_class)
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = Image.open(image_path)

        # Step 1: Check for blur
        if is_blurry(image):
            print(f"Skipping blurry image: {image_path}")
            continue
        
        # Step 2: Resize with padding to target size (256x256)
        image = ImageOps.fit(image, target_size, Image.LANCZOS)

        # Save processed image
        processed_image_path = os.path.join(output_folder_path, image_file)
        image.save(processed_image_path)

print("Preprocessing completed. Processed images are saved in the 'processed_data' folder.")


In [ ]:

from torch.utils.data import random_split

# Split dataset into training and validation for both non-preprocessed and preprocessed data
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Load the preprocessed dataset from the 'processed_data' folder
preprocessed_dataset = FruitDataset(root_dir='processed_data', classes=classes, transform=transform)
train_size_preprocessed = int(0.8 * len(preprocessed_dataset))
val_size_preprocessed = len(preprocessed_dataset) - train_size_preprocessed
train_dataset_preprocessed, val_dataset_preprocessed = random_split(preprocessed_dataset, [train_size_preprocessed, val_size_preprocessed])


In [ ]:

# Define the dataset with the original (non-preprocessed) images
dataset = FruitDataset(root_dir='train_data', classes=classes, transform=transform)


In [ ]:

from torch.utils.data import random_split

# Split dataset into training and validation for both non-preprocessed and preprocessed data
# Assuming 'dataset' is loaded for non-preprocessed data
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Load the preprocessed dataset from the 'processed_data' folder
preprocessed_dataset = FruitDataset(root_dir='processed_data', classes=classes, transform=transform)
train_dataset_preprocessed, val_dataset_preprocessed = random_split(preprocessed_dataset, [train_size, val_size])


In [8]:
from torch.utils.data import Subset
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import random_split

data_root =  './train_data/'
image_size = (300,300)

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

train_dataset = ImageFolder(root=data_root, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size

train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Baseline Model

In [9]:
# Step 3: Build a simple Baseline model.
# You should build a simple/standard neural network, i.e. multilayer perceptron (MLP) trained
# by back-propagation for this step, which helps you have a baseline intuition / idea of the task.
# It is probably easiest to just do this in PyTorch. Don’t use a CNN for this part! The point
# is to get a sense of how well a “vanilla” neural net can do, without convolutions. Because it
# is “dense”, this probably can’t be too large, as it has to be trained in a reasonable amount of
# time. Start small and see how you go.

class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
input_size = 300*300*3
hidden_size = 100
num_classes = 3

model = MLP(input_size, hidden_size, num_classes).to(device)

#Train the model
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum =0.9)

num_epochs = 10
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 300*300*3).to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/10], Loss: 0.9500730037689209
Epoch [2/10], Loss: 1.172192931175232
Epoch [3/10], Loss: 1.1247400045394897
Epoch [4/10], Loss: 0.7774066925048828
Epoch [5/10], Loss: 1.0034619569778442
Epoch [6/10], Loss: 0.9922547340393066
Epoch [7/10], Loss: 1.3390343189239502
Epoch [8/10], Loss: 0.7444821000099182
Epoch [9/10], Loss: 0.7488672137260437
Epoch [10/10], Loss: 0.6170305609703064


In [10]:
# Evaluate the Baseline model.
# Print the train and test accuracy
model.eval()
with torch.no_grad():
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images = images.reshape(-1, 300*300*3).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    print(f'Training Accuracy: {100 * correct_train / total_train}%')

    correct_test = 0
    total_test = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 300*300*3).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct_test / total_test}%')

Training Accuracy: 75.25083612040133%
Test Accuracy: 48.60646599777034%


In [ ]:

import matplotlib.pyplot as plt

# Function to train the model and return accuracy
def train_model(data_loader):
    model = MLPModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in data_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    return model

# Evaluate function
def evaluate_model(model, val_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train and evaluate on non-preprocessed data
train_loader_non_preprocessed = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader_non_preprocessed = DataLoader(val_dataset, batch_size=32, shuffle=False)
model_non_preprocessed = train_model(train_loader_non_preprocessed)
accuracy_non_preprocessed = evaluate_model(model_non_preprocessed, val_loader_non_preprocessed)

# Train and evaluate on preprocessed data
train_loader_preprocessed = DataLoader(train_dataset_preprocessed, batch_size=32, shuffle=True)
val_loader_preprocessed = DataLoader(val_dataset_preprocessed, batch_size=32, shuffle=False)
model_preprocessed = train_model(train_loader_preprocessed)
accuracy_preprocessed = evaluate_model(model_preprocessed, val_loader_preprocessed)

# Plotting the accuracy comparison
labels = ['Non-Preprocessed', 'Preprocessed']
accuracies = [accuracy_non_preprocessed, accuracy_preprocessed]
plt.bar(labels, accuracies, color=['blue', 'green'])
plt.xlabel("Dataset")
plt.ylabel("Accuracy (%)")
plt.title("Accuracy Comparison: Preprocessed vs Non-Preprocessed Data")
plt.show()
